In [1]:
import subprocess
import json

def test_load_data_from_s3_with_cli(bucket_name, file_key):
    try:
        # AWS CLI 명령을 사용하여 S3에서 파일 가져오기
        cmd = f"aws s3 cp s3://{bucket_name}/{file_key} -"
        result = subprocess.run(cmd, shell=True, capture_output=True, text=True)

        if result.returncode != 0:
            print(f"파일 다운로드 실패: {result.stderr}")
            return None
        
        data = result.stdout
        embedded_data = json.loads(data)
        
        # JSON 데이터 확인
        print(f"S3에서 로드한 데이터의 타입: {type(embedded_data)}")
        if isinstance(embedded_data, list):
            print(f"리스트의 길이: {len(embedded_data)}")
            for idx, item in enumerate(embedded_data):
                if not isinstance(item, dict):
                    print(f"인덱스 {idx}에서 비정상적인 데이터 발견: {item}")
                else:
                    missing_keys = [key for key in ['임베딩', '제목', '요약', '세부인정사항'] if key not in item]
                    if missing_keys:
                        print(f"인덱스 {idx}에서 누락된 키 발견: {missing_keys}")
                    else:
                        print(f"인덱스 {idx}의 임베딩 길이: {len(item['임베딩'])}")
                        if len(item['임베딩']) != 1536:
                            print(f"인덱스 {idx}의 임베딩 길이가 예상과 다릅니다.")
        else:
            print("embedded_data가 리스트가 아닙니다.")
        
        return embedded_data
    except json.JSONDecodeError as e:
        print(f"JSON 디코딩 오류: {e}")
        return None

# 테스트 실행
bucket_name = "hemochat-rag-database"
file_key = "18_aga_tagged_embedded_data.json"
embedded_data = test_load_data_from_s3_with_cli(bucket_name, file_key)


S3에서 로드한 데이터의 타입: <class 'list'>
리스트의 길이: 157
인덱스 0의 임베딩 길이: 1536
인덱스 1의 임베딩 길이: 1536
인덱스 2의 임베딩 길이: 1536
인덱스 3의 임베딩 길이: 1536
인덱스 4의 임베딩 길이: 1536
인덱스 5의 임베딩 길이: 1536
인덱스 6의 임베딩 길이: 1536
인덱스 7의 임베딩 길이: 1536
인덱스 8의 임베딩 길이: 1536
인덱스 9의 임베딩 길이: 1536
인덱스 10의 임베딩 길이: 1536
인덱스 11의 임베딩 길이: 1536
인덱스 12의 임베딩 길이: 1536
인덱스 13의 임베딩 길이: 1536
인덱스 14의 임베딩 길이: 1536
인덱스 15의 임베딩 길이: 1536
인덱스 16의 임베딩 길이: 1536
인덱스 17의 임베딩 길이: 1536
인덱스 18의 임베딩 길이: 1536
인덱스 19의 임베딩 길이: 1536
인덱스 20의 임베딩 길이: 1536
인덱스 21의 임베딩 길이: 1536
인덱스 22의 임베딩 길이: 1536
인덱스 23의 임베딩 길이: 1536
인덱스 24의 임베딩 길이: 1536
인덱스 25의 임베딩 길이: 1536
인덱스 26의 임베딩 길이: 1536
인덱스 27의 임베딩 길이: 1536
인덱스 28의 임베딩 길이: 1536
인덱스 29의 임베딩 길이: 1536
인덱스 30의 임베딩 길이: 1536
인덱스 31의 임베딩 길이: 1536
인덱스 32의 임베딩 길이: 1536
인덱스 33의 임베딩 길이: 1536
인덱스 34의 임베딩 길이: 1536
인덱스 35의 임베딩 길이: 1536
인덱스 36의 임베딩 길이: 1536
인덱스 37의 임베딩 길이: 1536
인덱스 38의 임베딩 길이: 1536
인덱스 39의 임베딩 길이: 1536
인덱스 40의 임베딩 길이: 1536
인덱스 41의 임베딩 길이: 1536
인덱스 42의 임베딩 길이: 1536
인덱스 43의 임베딩 길이: 1536
인덱스 44의 임베딩 길이: 1536
인덱스 45의 임베딩 길이: 153

In [2]:
import numpy as np

def test_extract_vectors_and_metadata(embedded_data):
    vectors = []
    metadatas = []
    
    if not isinstance(embedded_data, list):
        print("임베딩 데이터가 리스트 형식이 아닙니다.")
        return [], []
    
    for idx, item in enumerate(embedded_data):
        if isinstance(item, dict):
            if all(key in item for key in ['임베딩', '제목', '요약', '세부인정사항']):
                try:
                    vectors.append(np.array(item['임베딩']))
                    metadatas.append({
                        "제목": item["제목"],
                        "요약": item["요약"],
                        "세부인정사항": item["세부인정사항"]
                    })
                except (TypeError, ValueError) as e:
                    print(f"임베딩 데이터를 배열로 변환하는 중 오류 발생 (인덱스 {idx}): {e}")
            else:
                missing_keys = [key for key in ['임베딩', '제목', '요약', '세부인정사항'] if key not in item]
                print(f"인덱스 {idx}에서 누락된 키 발견: {missing_keys}")
        else:
            print(f"비정상적인 데이터 형식의 아이템 발견 (인덱스 {idx}): {item}")
    
    print(f"추출된 벡터의 수: {len(vectors)}")
    print(f"추출된 메타데이터의 수: {len(metadatas)}")
    
    return vectors, metadatas

# 테스트 실행
if embedded_data:
    vectors, metadatas = test_extract_vectors_and_metadata(embedded_data)


추출된 벡터의 수: 157
추출된 메타데이터의 수: 157


In [3]:
from sklearn.metrics.pairwise import cosine_similarity

def test_find_top_n_similar(embedding, vectors, metadatas, top_n=5):
    if len(vectors) != len(metadatas):
        print(f"벡터 수와 메타데이터 수가 일치하지 않습니다: 벡터 수 = {len(vectors)}, 메타데이터 수 = {len(metadatas)}")
        return []

    user_embedding = np.array(embedding).reshape(1, -1)
    similarities = cosine_similarity(user_embedding, vectors).flatten()
    
    print(f"코사인 유사도 배열의 길이: {len(similarities)}")
    
    top_indices = similarities.argsort()[-top_n:][::-1]
    print(f"상위 인덱스: {top_indices}")
    
    top_results = [{"유사도": similarities[i], "메타데이터": metadatas[i]} for i in top_indices]
    return top_results

# 테스트 실행
sample_embedding = np.random.rand(1536)  # 1536 차원 랜덤 벡터 생성
if vectors and metadatas:
    test_find_top_n_similar(sample_embedding, vectors, metadatas)


코사인 유사도 배열의 길이: 157
상위 인덱스: [116  55 127 148   7]


In [6]:
import openai

openai.api_key = "your-api-key"

def test_evaluate_relevance_with_gpt(user_input, items):
    try:
        prompt_template = "사용자 입력: {user_input}\n\n항목:\n{items}"
        formatted_items = "\n\n".join([f"항목 {i+1}: {item['요약']}" for i, item in enumerate(items)])
        prompt = prompt_template.format(user_input=user_input, items=formatted_items)
        
        print("프롬프트:", prompt)
        
        response = openai.ChatCompletion.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ]
        )
        
        result = response.choices[0].message.content.strip()
        print("GPT 응답:", result)
        
        return result
    except Exception as e:
        print(f"evaluate_relevance_with_gpt 호출 중 오류 발생: {e}")

# 테스트 실행
test_evaluate_relevance_with_gpt("테스트 입력", [{"제목": "테스트", "요약": "테스트 요약", "세부인정사항": "테스트 인정사항"}])


프롬프트: 사용자 입력: 테스트 입력

항목:
항목 1: 테스트 요약
GPT 응답: 테스트 입력에 대한 요약입니다:

**테스트 요약**:
- 입력된 내용: "테스트 입력"
- 목적: 사용자의 입력에 대한 브리핑이나 개요 제공

추가적으로 원하는 사항이나 구체적인 질문이 있다면 말씀해 주세요.


'테스트 입력에 대한 요약입니다:\n\n**테스트 요약**:\n- 입력된 내용: "테스트 입력"\n- 목적: 사용자의 입력에 대한 브리핑이나 개요 제공\n\n추가적으로 원하는 사항이나 구체적인 질문이 있다면 말씀해 주세요.'